In [1]:
import pandas as pd
import random
import sklearn
from sklearn.neighbors import NearestNeighbors
df = pd.read_csv("C:\\Users\\SUVARNA\\Food_recommendation\\Recommendation\\user_Profiles.csv")

In [2]:
Age = []
for i in range(df.shape[0]):
    Age.append(random.randint(10,90))
df.insert(1,"Age",Age,True)
#df.insert(2, "Age", [21, 23, 24, 21], True)
df.head()

,User_Id,Age,Veg_Non,Nutrient,Disease,Diet
0,User_1,78,non-veg,chloride,anemia kidney_disease goitre rickets pregnancy,type_a_diet high_protien_diet low_carb_diet l...
1,User_2,74,veg,chloride,goitre,high_fiber_diet vegan_diet
2,User_3,48,veg,magnesium,cancer hypertension goitre heart_disease scurvy,high_fiber_diet ketogenic_diet high_protien_diet
3,User_4,57,veg,vitamin_e,cancer kidney_disease obesity anemia heart_di...,high_fiber_diet ketogenic_diet gluten_free_di...
4,User_5,77,non-veg,vitamin_c,anemia kidney_disease goitre rickets pregnancy,type_a_diet high_protien_diet low_carb_diet l...


In [3]:
Height = []
for i in range(df.shape[0]):
    x = random.uniform(4,7)
    Height.append(round(x,2))
df.insert(2,"Height",Height,True)
#df.insert(2, "Age", [21, 23, 24, 21], True)
df.head()

,User_Id,Age,Height,Veg_Non,Nutrient,Disease,Diet
0,User_1,78,5.27,non-veg,chloride,anemia kidney_disease goitre rickets pregnancy,type_a_diet high_protien_diet low_carb_diet l...
1,User_2,74,4.22,veg,chloride,goitre,high_fiber_diet vegan_diet
2,User_3,48,6.99,veg,magnesium,cancer hypertension goitre heart_disease scurvy,high_fiber_diet ketogenic_diet high_protien_diet
3,User_4,57,6.29,veg,vitamin_e,cancer kidney_disease obesity anemia heart_di...,high_fiber_diet ketogenic_diet gluten_free_di...
4,User_5,77,4.05,non-veg,vitamin_c,anemia kidney_disease goitre rickets pregnancy,type_a_diet high_protien_diet low_carb_diet l...


In [4]:
class Recommender:
    
    def __init__(self,df):
        self.df = df
        self.activity = pd.read_csv('C:\\Users\\SUVARNA\\Food_recommendation\\DJANGO\\minor\\website\\csvfile\\user_activity.csv')
    
    def get_features(self):
        #getting dummies of dataset
        nutrient_dummies = self.df.Nutrient.str.get_dummies()
        disease_dummies = self.df.Disease.str.get_dummies(sep=' ')
        diet_dummies = self.df.Diet.str.get_dummies(sep=' ')
        feature_df = pd.concat([nutrient_dummies,disease_dummies,diet_dummies,df['Age'],df['Height']],axis=1)
     
        return feature_df
    
    def k_neighbor(self,inputs):
        
        feature_df = self.get_features()
        
        #initializing model with k=20 neighbors
        model = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
        
        # fitting model with dataset features
        model.fit(feature_df)
    
        
        df_results = pd.DataFrame(columns=list(self.df.columns))
        
      
        # getting distance and indices for k nearest neighbor
        distnaces , indices = model.kneighbors(inputs)

        for i in list(indices):
            df_results = df_results.append(self.df.loc[i])
                
#         df_results = df_result
        
        df_results = df_results.reset_index(drop=True)
        return df_results
    
    def user_based(self,user_profile,user_id):
        
        data = self.get_features()
        total_features = data.columns
        d = dict()
        for i in total_features:
            d[i]= 0
        for i in user_profile:
            d[i] = 1
        final_input = list(d.values())
        
        similar_users = self.k_neighbor([final_input])
        users = list(similar_users.User_Id)
        results = self.activity[self.activity.User_Id.isin(users)] #taking acitivies
        results = results[results['User_Id']!=user_id] # selecting those which are not reviewed by user
        results = results.filter(['Name','Nutrient','Veg_Non','Price','Review','Diet','Disease','description'])

        results = results.drop_duplicates(subset=['Name'])
        results = results.reset_index(drop=True)
        return results

In [5]:
ob = Recommender(df)
data = ob.get_features()

total_features = data.columns
d = dict()
for i in total_features:
    d[i]= 0
print(d)

{'calcium': 0, 'carbohydrates': 0, 'chloride': 0, 'fiber': 0, 'iron': 0, 'magnesium': 0, 'manganese': 0, 'phosphorus': 0, 'potassium': 0, 'protien': 0, 'selenium': 0, 'sodium': 0, 'vitamin_a': 0, 'vitamin_c': 0, 'vitamin_e': 0, 'anemia': 0, 'cancer': 0, 'diabeties': 0, 'eye_disease': 0, 'goitre': 0, 'heart_disease': 0, 'hypertension': 0, 'kidney_disease': 0, 'obesity': 0, 'pregnancy': 0, 'rickets': 0, 'scurvy': 0, 'Mediterranean_diet': 0, 'alkaline_diet': 0, 'dash_diet': 0, 'gluten_free_diet': 0, 'high_fiber_diet': 0, 'high_protien_diet': 0, 'hormone_diet': 0, 'ketogenic_diet': 0, 'low_carb_diet': 0, 'low_fat_diet': 0, 'low_sodium_diet': 0, 'omni_diet': 0, 'paleo_diet': 0, 'type_a_diet': 0, 'type_o_diet': 0, 'vegan_diet': 0, 'Age': 0, 'Height': 0}


In [6]:
profile_A = ['calcium','iron','diabeties','gluten_free_diet',56,5.6]
user_id = 'User_71' # asumming for example
sample_input = profile_A

for i in sample_input:
    if type(i)==int:
        d['Age'] = i
    elif type(i)==float:
        d['Height']=i
    else:
        d[i] = 1

final_input = list(d.values())
print(d)

{'calcium': 1, 'carbohydrates': 0, 'chloride': 0, 'fiber': 0, 'iron': 1, 'magnesium': 0, 'manganese': 0, 'phosphorus': 0, 'potassium': 0, 'protien': 0, 'selenium': 0, 'sodium': 0, 'vitamin_a': 0, 'vitamin_c': 0, 'vitamin_e': 0, 'anemia': 0, 'cancer': 0, 'diabeties': 1, 'eye_disease': 0, 'goitre': 0, 'heart_disease': 0, 'hypertension': 0, 'kidney_disease': 0, 'obesity': 0, 'pregnancy': 0, 'rickets': 0, 'scurvy': 0, 'Mediterranean_diet': 0, 'alkaline_diet': 0, 'dash_diet': 0, 'gluten_free_diet': 1, 'high_fiber_diet': 0, 'high_protien_diet': 0, 'hormone_diet': 0, 'ketogenic_diet': 0, 'low_carb_diet': 0, 'low_fat_diet': 0, 'low_sodium_diet': 0, 'omni_diet': 0, 'paleo_diet': 0, 'type_a_diet': 0, 'type_o_diet': 0, 'vegan_diet': 0, 'Age': 56, 'Height': 5.6}


In [7]:
similar_users = ob.k_neighbor([final_input]) # pass 2d array []
similar_users

c:\Users\SUVARNA\AppData\Local\Programs\Python\Python3.9\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,User_Id,Age,Height,Veg_Non,Nutrient,Disease,Diet
0,User_26,57,5.81,veg,iron,hypertension,high_fiber_diet gluten_free_diet vegan_diet l...
1,User_87,57,4.75,veg,vitamin_c,scurvy goitre anemia kidney_disease,high_fiber_diet dash_diet ketogenic_diet type...
2,User_4,57,6.29,veg,vitamin_e,cancer kidney_disease obesity anemia heart_di...,high_fiber_diet ketogenic_diet gluten_free_di...
3,User_49,58,6.66,veg,fiber,obesity diabeties goitre hypertension,high_protien_diet high_fiber_diet ketogenic_d...
4,User_17,59,6.52,veg,vitamin_e,cancer kidney_disease obesity anemia heart_di...,high_fiber_diet ketogenic_diet gluten_free_di...
5,User_94,59,4.57,non-veg,iron,anemia kidney_disease goitre rickets pregnancy,type_a_diet high_protien_diet high_fiber_diet...
